In [2]:
import os
import fnmatch
import pandas as pd 
import numpy as np
import math
import matplotlib.pyplot as plt
import networkx as nx
import pylab as palt
import shutil
import pickle
from statistics import mean
import statistics
from scipy.stats import skew
import heapq
from scipy.signal import argrelextrema
from numpy import mean, absolute
from scipy.stats import kurtosis
from scipy import stats 
import scipy.signal as sig
from pandas import DataFrame
import json
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

In [18]:
def convertFileIntoDictonary(file_path,isAlcoholic,jsonData):
    for file in os.listdir(file_path):
        data=pd.read_csv('{}/{}'.format(file_path,file))
        channelInfo=dict()
        for index, row in data.iterrows():
            if row["Sensor position"] not in channelInfo: 
                channelInfo[row["Sensor position"]]=[row["Sensor val"]]
            else:
                channelInfo[row["Sensor position"]].append(row["Sensor val"])
        jsonList = {'file name': file, 'raw signal data': channelInfo, 'connection info': '','nodes features info': '',
                            'edges feature info': '','pearson Correlation': '','Granger Causality':'','PLV':'','Partial Correlation': '','PLV' :'','isAlcoholic': isAlcoholic}        
        jsonData.append(jsonList)

In [19]:
def meanAbsoluteDeviation(jsonData):
    """
    Mean Absolute Deviation (M.A.D)
    """
    
    for data in jsonData:
        featureDict=dict()
        for key in data['raw signal data'].keys():
            if key not in featureDict:
                featureDict[key]=[np.mean(absolute(data['raw signal data'][key] - np.mean(data['raw signal data'][key])))]
            else:
                featureDict[key].append(np.mean(absolute(data['raw signal data'][key] - np.mean(data['raw signal data'][key]))))
        data['nodes feature info']=featureDict

In [20]:
def mean(jsonData):
    """
        mean
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(np.mean(data['raw signal data'][key]))

In [21]:
def zeroCrossing(jsonData):
    """
    No of time the signal crosses zero within an analysis window.
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            count=0
            for k in range(0,len(data['raw signal data'][key])-1):
                if k==0:
                    if (data['raw signal data'][key][k]>data['raw signal data'][key][k+1]):
                        count=count+1
                elif k==len(data['raw signal data'][key])-1:
                    if (data['raw signal data'][key][k]>data['raw signal data'][key][k-1]):
                        count=count+1
                else:
                    if (data['raw signal data'][key][k]>data['raw signal data'][key][k-1] and data['raw signal data'][key][k]>data['raw signal data'][key][k+1]) or (data['raw signal data'][key][k]<data['raw signal data'][key][k-1] and data['raw signal data'][key][k]<data['raw signal data'][key][k+1]):
                        count=count+1
            data['nodes feature info'][key].append(count)

In [22]:
def varience(jsonData):
    """
        varience
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(statistics.median(data['raw signal data'][key]))

In [23]:
def SD(jsonData):
    """
        varience
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(np.std(data['raw signal data'][key], dtype = np.float64))

In [24]:
def skewness(jsonData):
    """
        skewness
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(skew(data['raw signal data'][key]))

In [25]:
def kurtosisvalue(jsonData):
    """
        kurtosis
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(kurtosis(data['raw signal data'][key]))

In [26]:
def medianAbsoluteDeviation(jsonData):
    """
        median absolute deviation
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(stats.median_abs_deviation(data['raw signal data'][key]))

In [27]:
def minimum(jsonData):
    """
        Minimum
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(min(data['raw signal data'][key]))

In [28]:
def maximum(jsonData):
    """
        maximum
    """
    for data in jsonData:
        for key in data['raw signal data'].keys():
            data['nodes feature info'][key].append(max(data['raw signal data'][key]))

In [4]:
def hilphase(y1,y2):
    sig1_hill=sig.hilbert(y1)
    sig2_hill=sig.hilbert(y2)
    pdt=(np.inner(sig1_hill,np.conj(sig2_hill))/(np.sqrt(np.inner(sig1_hill,
               np.conj(sig1_hill))*np.inner(sig2_hill,np.conj(sig2_hill)))))
    phase = np.angle(pdt)

    return phase

In [5]:
def PLV(jsonData):
    """
        PLV
    """
    for data in jsonData:
        for key1 in data['raw signal data'].keys():
            for key2 in data['raw signal data'].keys():
                hilphase(data['raw signal data'][key1],data['raw signal data'][key2])

In [6]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [4]:
def pearsonCorrelation(jsonData): 
    for data in jsonData:    
        matrix=dict()
        for key1 in data['raw signal data'].keys():
            matrix_array=[]
            for key2 in data['raw signal data'].keys():
                x=data['raw signal data'][key1]
                y=data['raw signal data'][key2]
                r=np.corrcoef(x, y)
                matrix_array.append(r[0,1])
            matrix[key1]=matrix_array
        df = pd.DataFrame(matrix,columns=['FP1', 'FP2', 'F7', 'F8', 'AF1', 'AF2', 'FZ', 'F4', 'F3', 'FC6', 'FC5', 'FC2', 'FC1', 'T8', 'T7', 'CZ', 'C3', 'C4', 'CP5', 'CP6', 'CP1', 'CP2', 'P3', 'P4', 'PZ', 'P8', 'P7', 'PO2', 'PO1', 'O2', 'O1', 'X', 'AF7', 'AF8', 'F5', 'F6', 'FT7', 'FT8', 'FPZ', 'FC4', 'FC3', 'C6', 'C5', 'F2', 'F1', 'TP8', 'TP7', 'AFZ', 'CP3', 'CP4', 'P5', 'P6', 'C1', 'C2', 'PO7', 'PO8', 'FCZ', 'POZ', 'OZ', 'P2', 'P1', 'CPZ', 'nd', 'Y'])
        corrMatrix = df.corr()
        print(data['file name'])
        data['pearson Correlation']=corrMatrix

In [7]:
jsonData = []

In [32]:
path_alcohol="C:\\Users\\alvin\\Desktop\\MTP_Project\\data\\matching condition data\\alcohol"
convertFileIntoDictonary(path_alcohol,'Y',jsonData)

In [33]:
path_non_alcohol="C:\\Users\\alvin\\Desktop\\MTP_Project\\data\\matching condition data\\nonalcohol"
convertFileIntoDictonary(path_non_alcohol,'N',jsonData)

In [8]:
f = open('data.txt',)  
# returns JSON object as  
# a dictionary 
jsonData = json.load(f) 

In [7]:
column_values = np.array(jsonData[0]['raw signal data'].keys())
print(column_values)

dict_keys(['FP1', 'FP2', 'F7', 'F8', 'AF1', 'AF2', 'FZ', 'F4', 'F3', 'FC6', 'FC5', 'FC2', 'FC1', 'T8', 'T7', 'CZ', 'C3', 'C4', 'CP5', 'CP6', 'CP1', 'CP2', 'P3', 'P4', 'PZ', 'P8', 'P7', 'PO2', 'PO1', 'O2', 'O1', 'X', 'AF7', 'AF8', 'F5', 'F6', 'FT7', 'FT8', 'FPZ', 'FC4', 'FC3', 'C6', 'C5', 'F2', 'F1', 'TP8', 'TP7', 'AFZ', 'CP3', 'CP4', 'P5', 'P6', 'C1', 'C2', 'PO7', 'PO8', 'FCZ', 'POZ', 'OZ', 'P2', 'P1', 'CPZ', 'nd', 'Y'])


In [34]:
meanAbsoluteDeviation(jsonData)

In [35]:
mean(jsonData)

In [36]:
zeroCrossing(jsonData)

In [37]:
varience(jsonData)

In [38]:
SD(jsonData)

In [39]:
skewness(jsonData)

In [40]:
kurtosisvalue(jsonData)

In [41]:
medianAbsoluteDeviation(jsonData)

In [42]:
minimum(jsonData)

In [43]:
maximum(jsonData)

In [9]:
keys_List=jsonData[0]['raw signal data'].keys()

In [12]:
for data in jsonData:
    List=[]
    for key in data['raw signal data'].keys():
        List.append(data['raw signal data'][key])
    print(List)
    df=DataFrame(List).transpose()
    df.columns=keys_List
    grangers_causation_matrix(df, variables = df.columns) 

[[-8.921, -8.433, -2.574, 5.239, 11.587, 14.027999999999999, 11.587, 6.704, 1.821, -1.109, -2.085, -1.597, 0.35600000000000004, 2.309, 2.7969999999999997, 2.309, 0.35600000000000004, -0.132, -0.132, 0.35600000000000004, 0.8440000000000001, 1.3330000000000002, 2.7969999999999997, 6.215, 9.633, 10.61, 6.704, -0.621, -5.992000000000001, -6.48, -1.597, 4.262, 7.68, 5.727, 1.821, 0.35600000000000004, 3.286, 9.145, 14.027999999999999, 13.54, 8.169, 1.821, -1.109, 2.309, 9.633, 16.469, 18.910999999999998, 15.004000000000001, 8.169, 2.309, 1.3330000000000002, 4.262, 9.145, 11.587, 9.633, 5.727, 3.286, 4.751, 10.122, 15.004000000000001, 14.515999999999998, 8.169, -1.109, -8.433, -10.386, -7.456, -3.062, 0.35600000000000004, 0.8440000000000001, -0.132, -2.085, -3.062, -3.55, -3.062, -2.574, -2.574, -3.55, -5.502999999999999, -7.456, -7.945, -6.48, -4.038, -1.597, -0.132, -0.132, -0.132, 1.3330000000000002, 2.309, 2.309, 0.35600000000000004, -4.038, -8.433, -9.898, -8.921, -5.502999999999999, -2.

KeyboardInterrupt: 